In [1]:
import requests
import time
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

In [2]:
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [3]:
df = pd.read_csv('countries.csv', encoding = "utf-16")

In [4]:
df[df['city'] == 'London']['lat']

0    51.507322
Name: lat, dtype: float64

In [5]:

'''url = "https://air-quality-api.open-meteo.com/v1/air-quality"

# Number of retry attempts for a failed API call
max_retries = 3
retry_delay = 2  # Seconds to wait before retrying

for c in df['city']:
    params = {
        "latitude": df[df['city'] == c]['lat'].values[0],  # Ensure you fetch the value correctly
        "longitude": df[df['city'] == c]['lon'].values[0],
        "hourly": ["pm10", "pm2_5", "carbon_monoxide", "nitrogen_dioxide", "sulphur_dioxide", "ozone", "european_aqi"],
        "timezone": "auto",
        "start_date": "2020-01-01",
        "end_date": "2024-10-22"
    }

    success = False
    attempts = 0

    while not success and attempts < max_retries:
        try:
            # Sending the request with a timeout value (in seconds)
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()  # Raises an error for bad responses
            data = response.json()

            hourly_data = data.get('hourly', {})
            times = hourly_data.get('time', [])
            pm10_values = hourly_data.get('pm10', [])
            pm2_5_values = hourly_data.get('pm2_5', [])
            co_values = hourly_data.get('carbon_monoxide', [])
            no2_values = hourly_data.get('nitrogen_dioxide', [])
            so2_values = hourly_data.get('sulphur_dioxide', [])
            o3_values = hourly_data.get('ozone', [])
            eu_aqi_values = hourly_data.get('european_aqi', [])

            # Check if we have valid data before proceeding
            if times and pm10_values and pm2_5_values and co_values and no2_values and so2_values and o3_values and eu_aqi_values:
                # Create a pandas DataFrame for the hourly data
                hourly_df = pd.DataFrame({
                    "date": pd.to_datetime(times),
                    "pm10": pm10_values,
                    "pm2_5": pm2_5_values,
                    "carbon_monoxide": co_values,
                    "nitrogen_dioxide": no2_values,
                    "sulphur_dioxide": so2_values,
                    "ozone": o3_values,
                    "european_aqi": eu_aqi_values
                })
                
                print(f"Data for {c}:")
                print(hourly_df)
            else:
                print(f"No valid hourly data available for {c}")
            
            success = True  # If we get here, the request was successful

        except requests.exceptions.Timeout:
            # If a timeout occurs, print a message and retry
            attempts += 1
            print(f"Timeout occurred for {c}, attempt {attempts}/{max_retries}")
            time.sleep(retry_delay)  # Wait before retrying

        except requests.exceptions.RequestException as e:
            # For any other request exception, print an error and skip the city
            print(f"Error occurred for {c}: {e}")
            break  # Exit retry loop for this city
'''


'url = "https://air-quality-api.open-meteo.com/v1/air-quality"\n\n# Number of retry attempts for a failed API call\nmax_retries = 3\nretry_delay = 2  # Seconds to wait before retrying\n\nfor c in df[\'city\']:\n    params = {\n        "latitude": df[df[\'city\'] == c][\'lat\'].values[0],  # Ensure you fetch the value correctly\n        "longitude": df[df[\'city\'] == c][\'lon\'].values[0],\n        "hourly": ["pm10", "pm2_5", "carbon_monoxide", "nitrogen_dioxide", "sulphur_dioxide", "ozone", "european_aqi"],\n        "timezone": "auto",\n        "start_date": "2020-01-01",\n        "end_date": "2024-10-22"\n    }\n\n    success = False\n    attempts = 0\n\n    while not success and attempts < max_retries:\n        try:\n            # Sending the request with a timeout value (in seconds)\n            response = requests.get(url, params=params, timeout=10)\n            response.raise_for_status()  # Raises an error for bad responses\n            data = response.json()\n\n            hour

In [6]:
'''import requests
import pandas as pd
import time

url = "https://air-quality-api.open-meteo.com/v1/air-quality"

# Number of retry attempts for a failed API call
max_retries = 3
retry_delay = 2  # Seconds to wait before retrying

# List to store the results
all_data = []  # Will store each country's data as DataFrames

# Assuming df is your DataFrame with columns 'city', 'lat', 'lon'
for c in df['city']:
    params = {
        "latitude": df[df['city'] == c]['lat'].values[0],  # Ensure you fetch the value correctly
        "longitude": df[df['city'] == c]['lon'].values[0],
        "hourly": ["pm10", "pm2_5", "carbon_monoxide", "nitrogen_dioxide", "sulphur_dioxide", "ozone", "european_aqi"],
        "timezone": "auto",
        "start_date": "2020-01-01",
        "end_date": "2024-10-22"
    }

    success = False
    attempts = 0

    while not success and attempts < max_retries:
        try:
            # Sending the request with a timeout value (in seconds)
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()  # Raises an error for bad responses
            data = response.json()  # Convert response to JSON format

            # Extract the hourly data from the JSON response
            hourly_data = data.get('hourly', {})
            times = hourly_data.get('time', [])
            pm10_values = hourly_data.get('pm10', [])
            pm2_5_values = hourly_data.get('pm2_5', [])
            co_values = hourly_data.get('carbon_monoxide', [])
            no2_values = hourly_data.get('nitrogen_dioxide', [])
            so2_values = hourly_data.get('sulphur_dioxide', [])
            o3_values = hourly_data.get('ozone', [])
            eu_aqi_values = hourly_data.get('european_aqi', [])

            # Check if we have valid data before proceeding
            if all([times, pm10_values, pm2_5_values, co_values, no2_values, so2_values, o3_values, eu_aqi_values]):
                # Create a pandas DataFrame for the hourly data
                hourly_df = pd.DataFrame({
                    "date": pd.to_datetime(times),
                    "pm10": pm10_values,
                    "pm2_5": pm2_5_values,
                    "carbon_monoxide": co_values,
                    "nitrogen_dioxide": no2_values,
                    "sulphur_dioxide": so2_values,
                    "ozone": o3_values,
                    "european_aqi": eu_aqi_values
                })
                
                hourly_df['city'] = c  # Add city name as a column for later identification

                # Append the current city's data to the all_data list
                all_data.append(hourly_df)
                print(f"Data for {c}:")
                print(hourly_df)

            else:
                print(f"No valid hourly data available for {c}")
            
            success = True  # If the request was successful, break the retry loop

        except requests.exceptions.Timeout:
            # If a timeout occurs, print a message and retry
            attempts += 1
            print(f"Timeout occurred for {c}, attempt {attempts}/{max_retries}")
            time.sleep(retry_delay)  # Wait before retrying

        except requests.exceptions.RequestException as e:
            # For any other request exception, print an error and log the failure
            print(f"Error occurred for {c}: {e}")
            break  # Exit retry loop for this city, but continue with the next city

# After all cities are processed, combine all DataFrames into a single DataFrame
if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)
else:
    print("No data was collected.")
'''

'import requests\nimport pandas as pd\nimport time\n\nurl = "https://air-quality-api.open-meteo.com/v1/air-quality"\n\n# Number of retry attempts for a failed API call\nmax_retries = 3\nretry_delay = 2  # Seconds to wait before retrying\n\n# List to store the results\nall_data = []  # Will store each country\'s data as DataFrames\n\n# Assuming df is your DataFrame with columns \'city\', \'lat\', \'lon\'\nfor c in df[\'city\']:\n    params = {\n        "latitude": df[df[\'city\'] == c][\'lat\'].values[0],  # Ensure you fetch the value correctly\n        "longitude": df[df[\'city\'] == c][\'lon\'].values[0],\n        "hourly": ["pm10", "pm2_5", "carbon_monoxide", "nitrogen_dioxide", "sulphur_dioxide", "ozone", "european_aqi"],\n        "timezone": "auto",\n        "start_date": "2020-01-01",\n        "end_date": "2024-10-22"\n    }\n\n    success = False\n    attempts = 0\n\n    while not success and attempts < max_retries:\n        try:\n            # Sending the request with a timeout 

In [7]:
url = "https://air-quality-api.open-meteo.com/v1/air-quality"

# Number of retry attempts for each failed API call
max_retries = 3
retry_delay = 2  # Seconds to wait before retrying

# List to store the results
all_data = []  # Will store each city's data as DataFrames

# Set to track processed cities
processed_cities = set()

# List to track cities that failed after all retries
failed_cities = []

# Maximum number of total attempts for all cities
total_attempt_limit = 5

# Assuming df is your DataFrame with columns 'city', 'lat', 'lon'
def scrape_city(c):
    """Function to scrape a single city and return success or failure"""
    params = {
        "latitude": df[df['city'] == c]['lat'].values[0],  # Ensure you fetch the value correctly
        "longitude": df[df['city'] == c]['lon'].values[0],
        "hourly": ["pm10", "pm2_5", "carbon_monoxide", "nitrogen_dioxide", "sulphur_dioxide", "ozone", "european_aqi"],
        "timezone": "auto",
        "start_date": "2020-01-01",
        "end_date": "2024-10-22"
    }

    success = False
    attempts = 0

    while not success and attempts < max_retries:
        try:
            # Sending the request with a timeout value (in seconds)
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()  # Raises an error for bad responses
            data = response.json()  # Convert response to JSON format

            # Extract the hourly data from the JSON response
            hourly_data = data.get('hourly', {})
            times = hourly_data.get('time', [])
            pm10_values = hourly_data.get('pm10', [])
            pm2_5_values = hourly_data.get('pm2_5', [])
            co_values = hourly_data.get('carbon_monoxide', [])
            no2_values = hourly_data.get('nitrogen_dioxide', [])
            so2_values = hourly_data.get('sulphur_dioxide', [])
            o3_values = hourly_data.get('ozone', [])

            # Check if we have valid data before proceeding
            if all([times, pm10_values, pm2_5_values, co_values, no2_values, so2_values, o3_values]):
                # Create a pandas DataFrame for the hourly data
                hourly_df = pd.DataFrame({
                    "date": pd.to_datetime(times),
                    "pm10": pm10_values,
                    "pm2_5": pm2_5_values,
                    "carbon_monoxide": co_values,
                    "nitrogen_dioxide": no2_values,
                    "sulphur_dioxide": so2_values,
                    "ozone": o3_values,
                })
                hourly_df['city'] = c  # Add city name as a column for later identification
                

                # Append the current city's data to the all_data list
                all_data.append(hourly_df)
                print(hourly_df)

                success = True  # Mark as successful if everything is fine
                return True  # Return True if data collection is successful

            else:
                print(f"No valid hourly data available for {c}")

        except requests.exceptions.Timeout:
            # If a timeout occurs, print a message and retry
            attempts += 1
            print(f"Timeout occurred for {c}, attempt {attempts}/{max_retries}")
            time.sleep(retry_delay)  # Wait before retrying

        except requests.exceptions.RequestException as e:
            # For any other request exception, print an error
            print(f"Error occurred for {c}: {e}")
            break  # Exit retry loop for this city

    return False  # Return False if the city couldn't be processed after retries


# Initial pass for all cities
for c in df['city']:
    # Check if the city has already been processed
    if c in processed_cities:
        continue  # Skip this city if it's already been processed

    # Try scraping the city
    if scrape_city(c):
        processed_cities.add(c)  # Add to processed cities if successful
    else:
        failed_cities.append(c)  # Add to failed cities if unsuccessful

# Retry for failed cities up to `total_attempt_limit` times
attempts = 0
while failed_cities and attempts < total_attempt_limit:
    for c in failed_cities.copy():  # Use a copy of the list to modify it safely
        if scrape_city(c):
            processed_cities.add(c)  # Mark city as processed
            failed_cities.remove(c)  # Remove city from failed list if successful

    attempts += 1
# Combine all DataFrames into a single DataFrame if data is collected
if all_data:
    hourly_dataframe = pd.concat(all_data, ignore_index=True)
    hourly_dataframe = hourly_dataframe.merge(df[['city', 'country']], on = 'city', how = 'left')
else:
    print("No data was collected.")


                     date  pm10  pm2_5  carbon_monoxide  nitrogen_dioxide  \
0     2020-01-01 00:00:00   NaN    NaN              NaN               NaN   
1     2020-01-01 01:00:00  30.6   23.1            217.0              28.1   
2     2020-01-01 02:00:00  29.3   23.3            217.0              21.9   
3     2020-01-01 03:00:00  24.9   21.2            211.0              21.2   
4     2020-01-01 04:00:00  22.1   18.1            202.0              19.8   
...                   ...   ...    ...              ...               ...   
42163 2024-10-22 19:00:00  15.9   12.0            267.0              60.6   
42164 2024-10-22 20:00:00  19.1   14.8            274.0              56.9   
42165 2024-10-22 21:00:00  20.4   17.1            256.0              54.0   
42166 2024-10-22 22:00:00  23.0   18.3            264.0              52.5   
42167 2024-10-22 23:00:00  25.9   21.7            244.0              54.2   

       sulphur_dioxide  ozone    city  
0                  NaN    NaN  Lond

In [8]:
hourly_dataframe.to_csv('air_quality_data.csv', index=False, encoding = 'utf-16')

In [9]:
data = pd.read_csv('air_quality_data.csv', encoding = 'utf-16')

In [10]:
data.dropna(axis = 0)

,date,pm10,pm2_5,carbon_monoxide,nitrogen_dioxide,sulphur_dioxide,ozone,city,country
1,2020-01-01 01:00:00,30.6,23.1,217.0,28.1,2.8,14.0,London,United Kingdom
2,2020-01-01 02:00:00,29.3,23.3,217.0,21.9,3.1,21.0,London,United Kingdom
3,2020-01-01 03:00:00,24.9,21.2,211.0,21.2,2.2,27.0,London,United Kingdom
4,2020-01-01 04:00:00,22.1,18.1,202.0,19.8,2.5,28.0,London,United Kingdom
5,2020-01-01 05:00:00,18.8,16.8,196.0,16.2,2.2,31.0,London,United Kingdom
...,...,...,...,...,...,...,...,...,...
2066227,2024-10-22 19:00:00,14.1,9.0,129.0,15.3,5.6,73.0,Gibraltar,Gibraltar
2066228,2024-10-22 20:00:00,14.8,9.1,130.0,16.0,5.5,70.0,Gibraltar,Gibraltar
2066229,2024-10-22 21:00:00,15.6,9.1,132.0,16.1,5.3,72.0,Gibraltar,Gibraltar
2066230,2024-10-22 22:00:00,17.1,9.2,133.0,15.0,5.3,72.0,Gibraltar,Gibraltar
